
# IFCD104 – Tema 3: Callbacks de Keras (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger)

**Objetivo:** demostrar cómo **callbacks** ayudan a entrenar de manera más **estable, eficiente y reproducible**.


In [ ]:

import os, datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## 1) Datos

In [ ]:

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0
x_train = np.expand_dims(x_train, -1)
x_test  = np.expand_dims(x_test, -1)


## 2) Modelo CNN sencillo

In [ ]:

def make_cnn():
    inputs = keras.Input(shape=(28,28,1))
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs, name="cnn_callbacks_demo")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = make_cnn()


## 3) Definir callbacks

In [ ]:

import os
os.makedirs("checkpoints", exist_ok=True)
checkpoint_path = "checkpoints/best_model.keras"
csv_path = "training_log.csv"

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True),
    keras.callbacks.CSVLogger(csv_path)
]


## 4) Entrenamiento con callbacks

In [ ]:

history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)

print("Modelo guardado en:", checkpoint_path)


## 5) Curvas de entrenamiento

In [ ]:

plt.figure()
plt.plot(history.history['loss'], label='loss_train')
plt.plot(history.history['val_loss'], label='loss_val')
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Pérdida con callbacks")
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history['accuracy'], label='acc_train')
plt.plot(history.history['val_accuracy'], label='acc_val')
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.title("Accuracy con callbacks")
plt.legend()
plt.show()


## 6) Evaluación y carga del mejor modelo

In [ ]:

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Modelo en memoria -> test_acc={acc:.4f} | test_loss={loss:.4f}")

best = keras.models.load_model(checkpoint_path)
loss_b, acc_b = best.evaluate(x_test, y_test, verbose=0)
print(f"Mejor checkpoint -> test_acc={acc_b:.4f} | test_loss={loss_b:.4f}")


## 7) Log de entrenamiento (CSV)

In [ ]:

import pandas as pd
log = pd.read_csv("training_log.csv")
log.tail()
